In [2]:
import pickle
from datetime import timedelta, datetime
from tqdm import tqdm
import json
from collections import defaultdict, Counter
from sklearn.linear_model import LogisticRegression
from scipy import sparse
import numpy as np
import requests
import pandas as pd
from copy import copy
import scipy.stats as sts
from scipy.special import expit as sigmoid

## 1. Прочитайте и проанализируйте данные

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:

+ взять в тренировочный набор турниры с dateStart из 2019 года
+ в тестовый — турниры с dateStart из 2020 года

In [3]:
with open("data/tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)
    
with open("data/results.pkl", "rb") as f:
    results = pickle.load(f)  

In [4]:
with open('data/players.pkl', 'rb') as file:
    players = pickle.load(file)

In [5]:
date_train_start = datetime.strptime('2019-01-01', '%Y-%m-%d')
date_val_start = datetime.strptime('2020-01-01', '%Y-%m-%d')

tournaments_train = []
tournaments_val = []

max_player_id = 0
max_team_id = 0

for key in tqdm(tournaments.keys(), position=0, leave=False):
    tournament_date = datetime.fromisoformat(tournaments[key]['dateStart']).replace(tzinfo=None)
    if tournament_date >= date_train_start:
        
        # Если нету команд, то пропускаем
        if not results[key]:
            continue
        
        total_questions = set()
        for team in results[key]:
            if team.get('mask') is not None:
                total_questions.add(len(team['mask']))
        
        # Пропустим турниры, где разное число вопросов
        if len(total_questions) > 1:
            continue
            
        tournament = {}
        tournament['id'] = tournaments[key]['id']
        tournament['teams'] = []
        
        for team in results[key]:
            # Уберем команды, где нет ответов или в ответах есть 'X', '?'
            if team.get('mask') is None or team.get('mask').replace('1', '').replace('0', ''):
                continue
                
            if team['team']['id'] > max_team_id:
                max_team_id = team['team']['id']
            
            team_dict = dict()
            team_dict['id'] = team['team']['id']
            team_dict['mask'] = team['mask']
            team_dict['members'] = []
            
            for member in team['teamMembers']:
                
                player_id = member['player']['id']
                if player_id > max_player_id:
                    max_player_id = player_id
                    
                team_dict['members'].append(player_id)
                
            tournament['teams'].append(team_dict)
        
        if not tournament['teams']:
            continue
        
        # Сразу будем делить на трейн и валидацию
        if tournament_date < date_val_start:
            tournaments_train.append(tournament)
        else:
            tournaments_val.append(tournament)

## 2. Baseline-модель

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:

+ повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации
+ в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса
+ для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков

Построим что-то вроде индекса и обратного индекса для каждого игрока

In [9]:
members = set()
member_answers = defaultdict(int)
member_right_answers = defaultdict(int)
member_tours = defaultdict(int)

for tournament in tournaments_train:
    tournament_answers = []
    for team in tournament['teams']:
        for member in team['members']:
            member_answers[member] += len(team['mask'])
            member_right_answers[member] = sum(list(map(int, team['mask'])))
            member_tours[member] += 1
            members.add(member)
        

member_to_idx = {member:idx for idx, member in enumerate(members)}
idx_to_members = {idx:member for member, idx in member_to_idx.items()}
member_1000 = [member for member, answers in member_answers.items() if answers > 1000]

In [10]:
member_idxs = []
question_idxs = []
team_ids = []
tournament_ids = []
results = []
member_questions_count = []

questions_count = 0
for tournament in tqdm(tournaments_train, position=0, leave=False):
    tour_questions_count = len(tournament['teams'][0]['mask'])
    for team in tournament['teams']:
        team_answers = list(map(int, team['mask']))
        for q in range(tour_questions_count):
            for member in team['members']:
                member_idxs.append(member_to_idx[member])
                question_idxs.append(len(member_to_idx) + questions_count + q)
                team_ids.append(team['id'])
                tournament_ids.append(tournament['id'])
                results.append(team_answers[q])
                member_questions_count.append(member_answers[member])
    questions_count += tour_questions_count
    
    
X = sparse.lil_matrix((len(member_idxs), len(member_to_idx) + questions_count),  dtype=int)
X[range(len(member_idxs)), member_idxs] = 1
X[range(len(member_idxs)), question_idxs] = 1
y = np.array(results)

dim0, dim1 = X.shape

In [11]:
lr = LogisticRegression()

In [12]:
lr.fit(X, y)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
question_rating = lr.coef_[0][len(member_to_idx):]
rating = lr.coef_[0][:len(member_to_idx)]
rating_list = []
for idx, member in idx_to_members.items():
    item = {
        'score': rating[idx],
        'id': member,
        'name': players[member]['name'] + ' ' + players[member]['surname'],
        'questions_count': member_answers[member],
    }
    rating_list.append(item)
    
sorted_rating = sorted(rating_list, key=lambda x: x['score'], reverse=True)

In [16]:
def get_member_position(id):
    url = f'https://rating.chgk.info/api/players/{id}/rating/last'
    position = -1
    try:
        position = requests.get(url).json()['rating_position']
        position = int(position)
    except:
        pass
    
    return position

In [17]:
df_rating = pd.DataFrame(sorted_rating)[:50]
df_rating['actual_position'] = df_rating['id'].apply(get_member_position)
df_rating['questions_count'] = df_rating['id'].apply(lambda x: member_answers[x])
df_rating.head(25)

,score,id,name,questions_count,actual_position
0,3.811086,27403,Максим Руссо,1796,5
1,3.559784,4270,Александра Брутер,2240,6
2,3.430787,37047,Мария Юнгер,452,589
3,3.398291,30152,Артём Сорожкин,4006,1
4,3.316383,20691,Станислав Мереминский,1370,38
5,3.289878,28751,Иван Семушин,3071,3
6,3.286252,34328,Михаил Царёв,366,310
7,3.280127,27822,Михаил Савченков,2666,2
8,3.277498,3843,Светлана Бомешко,336,3926
9,3.239502,18036,Михаил Левандовский,1113,8


In [18]:
top_50_count = df_rating[df_rating['actual_position'] <= 100].shape[0]

print(f'В топ-50 рейтинга модели попали {top_50_count} игроков из топ-100 реального рейтинга')

В топ-50 рейтинга модели попали 27 игроков из топ-100 реального рейтинга


Уберем игроков, которые сыграли меньше 1000 вопросов.

In [19]:
df_rating_1000 = pd.DataFrame(list(filter(lambda x: x['id'] in member_1000, sorted_rating)))[:50]
df_rating_1000['actual_position'] = df_rating_1000['id'].apply(get_member_position)
df_rating_1000['questions_count'] = df_rating_1000['id'].apply(lambda x: member_answers[x])
df_rating_1000.head(25)

,score,id,name,questions_count,actual_position
0,3.811086,27403,Максим Руссо,1796,5
1,3.559784,4270,Александра Брутер,2240,6
2,3.398291,30152,Артём Сорожкин,4006,1
3,3.316383,20691,Станислав Мереминский,1370,38
4,3.289878,28751,Иван Семушин,3071,3
5,3.280127,27822,Михаил Савченков,2666,2
6,3.239502,18036,Михаил Левандовский,1113,8
7,3.222995,56647,Наталья Горелова,1769,349
8,3.200083,22935,Илья Новиков,1266,132
9,3.187588,30270,Сергей Спешков,3017,4


In [36]:
top_50_count = df_rating_1000[df_rating_1000['actual_position'] <= 50].shape[0]

print(f'В топ-50 рейтинга модели попали {top_50_count} игроков из топ-50 реального рейтинга')

В топ-50 рейтинга модели попали 28 игроков из топ-50 реального рейтинга


## 3. Оценка качества

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;

- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Для определения рейтинга команды воспользуемся вероятностной интерпретацией. Если рейтинг i-го игрока $r_i$ равен вероятности взять им случайный вопрос, тогда вероятность взять вопрос командой $T$ равна
$$
r_T = 1 - \prod_{i \in T} \left(1 - r_i\right).
$$

In [23]:
tournaments_test = []
tournaments_rating_true = []

for tournament in tournaments_val:
    teams_test = []
    teams_rating = []
    for team in tournament['teams']:
  
        memebers = [member for member in team['members'] if member in member_to_idx.keys()]
        team_test = copy(team)
        team_test['members'] = memebers
        
        if len(memebers) > 0:
            teams_test.append(team_test)
            team_answers = list(map(int, team['mask']))
            team_score = sum(team_answers)
            teams_rating.append(team_score)
     
    if len(teams_test) > 1:
        tournament_test = copy(tournament)
        tournament_test['teams'] = teams_test
        tournaments_test.append(tournament_test)
        tournaments_rating_true.append(teams_rating)

In [24]:
def predict_tournaments(model, tournaments, member_to_idx):

    tournaments_rating_pred = []
    for torunament in tqdm(tournaments, position=0, leave=False):
        tournament_questions_count = len(tournament['teams'][0]['mask'])
        preds = []
        for team in torunament['teams']:
            memeber_idxs = [member_to_idx[member] for member in team['members']]
            members_count = len(memeber_idxs)
 
            X = sparse.lil_matrix((members_count, dim1), dtype=int)
            X[range(len(memeber_idxs)), memeber_idxs] = 1
        
            fail_probas = model.predict_proba(X)[:, 0]
            team_proba = 1 - fail_probas.prod() 
            preds.append(team_proba)

        tournaments_rating_pred.append(preds)
    return tournaments_rating_pred

def print_correlations(tournaments_rating_true, tournaments_rating_pred):

    spearmanr_corrs = []
    kendall_corrs = []
    for i in range(len(tournaments_rating_true)):
        spearman = sts.spearmanr(tournaments_rating_true[i], tournaments_rating_pred[i]).correlation
        kendall = sts.kendalltau(tournaments_rating_true[i], tournaments_rating_pred[i]).correlation
        spearmanr_corrs.append(spearman)
        kendall_corrs.append(kendall)

    print(f'Корреляция Спирмена: {np.mean(spearmanr_corrs):.4f}')
    print(f'Корреляция Кендалла: {np.mean(kendall_corrs):.4f}')

In [27]:
tournaments_rating_pred = predict_tournaments(
    model=lr, 
    tournaments=tournaments_test,
    member_to_idx=member_to_idx, 
)

print_correlations(tournaments_rating_true, tournaments_rating_pred)

Корреляция Спирмена: 0.7731
Корреляция Кендалла: 0.6160


## 4. EM-алгоритм

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:

- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


В качестве вектора скрытых переменных будем использовать вероятность ответа игроком на вопрос при условии команды: $z = P(member=1|team)$

- E-шаг: предсказываем вероятности ответа на вопрос игрока при условии команды: 
$P(member=1|team=1) = \frac{P(member=1 \cap team=1)}{P(team=1)} = \frac{P(team=1 | member=1) P(member=1)}{P(team=1)} = \frac{P(member=1)}{P(team=1)}$
- M-шаг: максимизируем правдоподобие. Обучаем модель на вероятностях с E-шага

In [28]:
lr_initial = LogisticRegression()
lr_initial.fit(X, y)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [29]:
class ProbabilityEstimator:
    def __init__(self, init_classifier=None):
        if init_classifier:
            self.w = np.hstack((init_classifier.intercept_, init_classifier.coef_[0]))
        else:
            self.w = None
        
    def fit(self, X, y, max_iters=100000, lr=10, batch_size=1000, tol=0.0000001):
        X = sparse.hstack([np.ones(len(y)).reshape(-1, 1), X], format='csr')
        if self.w is None:
            self.w = np.random.normal(size=X.shape[1])
            
        losses = []
        prev_mean_loss = np.inf
        
        data_len = X.shape[0]
        
        for i in range(max_iters):
            batch_idxs = np.random.choice(data_len, batch_size)
            X_batch = X[batch_idxs, :]
            y_batch = y[batch_idxs]
            preds = sigmoid(X_batch.dot(self.w))
            losses.append(self.log_loss(y_batch, preds))
            if (i + 1) % 1000 == 0:
                new_mean_loss = np.mean(losses)
                if (prev_mean_loss - new_mean_loss) < tol:
                    print(f'Fitted. Mean NLL: {new_mean_loss}', flush=True)
                    break                    
                losses = []
                prev_mean_loss = new_mean_loss

            grad = -X_batch.T.dot(y_batch - preds) / len(y_batch)
            self.w -= lr * grad 
    
    def predict_proba(self, X):
        X = sparse.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
        preds = sigmoid(X.dot(self.w))
        return np.hstack(((1 - preds).reshape(-1, 1), preds.reshape(-1, 1)))
    
    @staticmethod
    def log_loss(y, p):
        return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p))

In [32]:
estimator = ProbabilityEstimator(init_classifier=lr_initial)

tournaments_rating_pred = predict_tournaments(
    model=estimator, 
    tournaments=tournaments_test,
    member_to_idx=member_to_idx, 
)
print_correlations(tournaments_rating_true, tournaments_rating_pred)

for _ in range(15):
    
    # E-шаг
    preds = estimator.predict_proba(X)
    
    # Вероятность ответа игрока при условии команды
    df_team = pd.DataFrame({'team': team_ids, 'question': question_idxs, 'fail_pred': preds[:, 0], 'success_pred': preds[:, 1]})
    df_team_pis = df_team.groupby(['team', 'question']).agg({'fail_pred': 'prod'}).reset_index()
    df_team_pis['team_success_pred'] = 1 - df_team_pis['fail_pred']
    df_team_pis.drop(columns=['fail_pred'], inplace=True)
    df_team = pd.merge(df_team, df_team_pis, left_on=['team', 'question'],  right_on=['team', 'question'])
    z = (df_team['success_pred'] / df_team['team_success_pred']).clip(0, 1)
    
    # По нашему предположению, мы уверены, что если команда не ответила на вопрос, 
    # то никто из команды на него не ответил
    z[y == 0] = 0
    
    
    # M-шаг
    estimator.fit(X, z, lr=20)

    tournaments_rating_pred = predict_tournaments(
        model=estimator, 
        tournaments=tournaments_test,
        member_to_idx=member_to_idx, 
    )
    print_correlations(tournaments_rating_true, tournaments_rating_pred)
    


Корреляция Спирмена: 0.7731
Корреляция Кендалла: 0.6160
Fitted. Mean NLL: 0.5184371289925075


Корреляция Спирмена: 0.7875
Корреляция Кендалла: 0.6293
Fitted. Mean NLL: 0.300716454245231


Корреляция Спирмена: 0.7949
Корреляция Кендалла: 0.6384
Fitted. Mean NLL: 0.3071429293076535


Корреляция Спирмена: 0.7888
Корреляция Кендалла: 0.6341
Fitted. Mean NLL: 0.31043349723709984


Корреляция Спирмена: 0.7900
Корреляция Кендалла: 0.6351
Fitted. Mean NLL: 0.3097325761839406


...

Корреляция Спирмена: 0.7937
Корреляция Кендалла: 0.6396
Fitted. Mean NLL: 0.3030840415709736


Корреляция Спирмена: 0.7953
Корреляция Кендалла: 0.6405
Fitted. Mean NLL: 0.309864576524212


Корреляция Спирмена: 0.7954
Корреляция Кендалла: 0.6412
Fitted. Mean NLL: 0.3074324884168998


Корреляция Спирмена: 0.7961
Корреляция Кендалла: 0.6416
Fitted. Mean NLL: 0.3126831552432646


Корреляция Спирмена: 0.7956
Корреляция Кендалла: 0.6411


Метрики и правда немного выросли

## 5. Рейтинг турниров по сложности вопросов

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [42]:
tournament_diffs = {}
for tournament in tournaments_train:
    sum = 0
    for team in tournament['teams']:
        sum += team['mask'].count('1') /  len(team['mask'])
    tournament_diffs[tournament['id']] = sum / len(tournament['teams'])

In [44]:
tournament_diffs = pd.Series(tournament_diffs).sort_values(ascending=False)

In [45]:
print('Самые легкие турниры:')
for i, tournament in enumerate(tournament_diffs.index[:10], 1):
    print(f'{i:2}. {tournaments[tournament]["name"]}')

Самые легкие турниры:
 1. Второй тематический турнир имени Джоуи Триббиани
 2. Асинхрон по South Park
 3. Первый турнир имени Джоуи Триббиани
 4. Регулярный чемпионат МГУ. Высшая лига. Третий игровой день
 5. Чемпионат МГУ. Высшая лига. Второй игровой день
 6. Асинхрон по «Королю и Шуту»
 7. Ничто, нигде, никогда
 8. Шестой киевский марафон. Асинхрон
 9. Кубок Закарпатья
10. Синхрон-lite. Выпуск XXX


In [48]:
print('Самые сложные турниры:')
for i, tournament in enumerate(tournament_diffs.index[-1:-11:-1], 1):
    print(f'{i:2}. {tournaments[tournament]["name"]}')

Самые сложные турниры:
 1. Чемпионат Санкт-Петербурга. Первая лига
 2. Чемпионат Таджикистана
 3. Чемпионат Туркменистана
 4. Открытый Студенческий чемпионат Краснодарского края
 5. Студенческий чемпионат Тюменской области
 6. Молодёжный чемпионат Нижегородской области
 7. Зимник
 8. Чемпионат Кыргызстана
 9. Угрюмый Ёрш
10. ТрЭК-13


В целом, рейтинг-лист турниров соответствует интуиции.